# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [1]:
# all import statements needed for the project, for example:

import math
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import re
import pyarrow.parquet as pq
import geopandas as gpd
import os
import datetime
from math import sin, cos, sqrt, atan2, radians

In [2]:
# any general notebook setup, like log formatting
import warnings
warnings.filterwarnings("ignore")

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uberdata/uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [4]:
# Create folder 'taxidata' because of .gitignore
try:
    os.mkdir("taxidata")
    os.mkdir("sql_files")

except FileExistsError:
            pass

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

### Calculating distance

The calculate_distance function calculates the approximate distance between pickup and drop off points using latitude and longitude coordinates. 

The add_distance_column function adds a column with the calculated distance values to the taxi data set to facilitate analysis going forward.   

Note that these functions only use the 'math' module fron the standard library 

In [5]:
def calculate_distance(start_lat, start_lon, end_lat, end_lon):
    """
    Calculates the approximate distance 
    between pickup and drop off points using 
    latitude and longitude coordinates.

    Arguments
    ----------
    start_lat:
        Starting lattitude.
    start_lon:
        Starting longtitude.
    end_lat:
        Ending lattitude
    end_lon:
        Ending longtitude

    Returns
    -------
    The calculated distance  
    """
    
    R = 6373.0

    lat1 = radians(start_lat)
    lon1 = radians(start_lon)
    lat2 = radians(end_lat)
    lon2 = radians(end_lon)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    
    return round(distance, 2)

In [6]:
def add_distance_column(df):
    """
    Adds distance column to a dataframe

    Arguments
    ----------
    df:
        Data frame.
  
    Returns
    -------
    The data frame with distance column column added 
    """
    
    distance = []
    
    for i in df.index:
        estimated_distance = calculate_distance(df["pickup_latitude"][i], df["pickup_longitude"][i], df["dropoff_latitude"][i], df["dropoff_longitude"][i])
        distance.append(estimated_distance)
        
    df["calculated_distance"] = distance
       
    return df

### Converting datetime

Function transforms date columns from strings to datetime Python objects to facilitate analysis going forward.   

In [7]:
def datetime_str_to_obj(date_time_str):
    """
    Transforms date columns from strings to datetime Python objects

    Arguments
    ----------
    date_time_str:
        Date time string.
  
    Returns
    -------
    Date time objects  
    """
    
    date_time_obj = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')

    return date_time_obj    

### Processing Taxi Data

Regex is used to find relevant URLs for yellow taxi files within specified timeframe and programmatically download the files. Functions are then created to clean data sets. For example, unwanted columns are dropped, datapoints outside the specified NYC area are removed, and column names are normalized. This is done to facilitate more efficient and effective analysis in later steps. Details are highlighted in code comments.

In [8]:
def find_taxi_csv_urls():
    """
    Uses regex to find all taxi URLs

    Returns
    -------
    All taxi links  
    """
    TAXI_URL = "https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
    
    response = requests.get(TAXI_URL)
    html = response.content
    data = []
    soup = bs4.BeautifulSoup(html, "html.parser")
    for link in soup("a"):
        name = link.get("title")
        date = link.get("href")
        date_pattern = r'201[012345]|2009'
        if name == "Yellow Taxi Trip Records" and re.search(date_pattern, date):
            if not re.search(r'2015\-0[789]|2015\-1[012]', date):
                data.append(link.get("href"))
    return data

In [9]:
def get_lat_lon_from_loc():
    find_lat_lon = gpd.read_file("taxi_zones/taxi_zones.shp")
  
    find_lat_lon = find_lat_lon.to_crs(4326)
    lon = find_lat_lon.centroid.x 
    lat = find_lat_lon.centroid.y
    find_lat_lon["lon"] = lon
    find_lat_lon["lat"] = lat

    return find_lat_lon

In [10]:
def filter_lat_lon(df):
    lon_border = [NEW_YORK_BOX_COORDS[0][1], NEW_YORK_BOX_COORDS[1][1]]
    lat_border = [NEW_YORK_BOX_COORDS[0][0], NEW_YORK_BOX_COORDS[1][0]]
    
    deleted_row = []
    
    for i in df.index:
        if df["pickup_longitude"][i] < lon_border[0] or df["pickup_longitude"][i] > lon_border[1] or df["dropoff_longitude"][i] < lon_border[0] or df["dropoff_longitude"][i] > lon_border[1]:
            deleted_row.append(i)
            
        elif df["pickup_latitude"][i] < lat_border[0] or df["pickup_latitude"][i] > lat_border[1] or df["dropoff_latitude"][i] < lat_border[0] or df["dropoff_latitude"][i] > lat_border[1]:
            deleted_row.append(i)
            
            
    df = df.drop(labels = deleted_row, axis=0)
    
    return df.reset_index(drop=True)

In [11]:
def clean_data(df):
    # Normalize column for 2009 data
    if "Passenger_Count" in df.columns :
        df.rename(columns={"Passenger_Count": 'passenger_count', "Fare_Amt": 'fare_amount', "Trip_Distance": 'trip_distance'}, inplace=True)
    
    deleted_row = []
    
    for i in df.index:        
        # Trips with zero passenger count
        if df["passenger_count"][i] < 1 or df["passenger_count"][i] == False:
            deleted_row.append(i)
            
        # Trips with no fare
        elif df["fare_amount"][i] <= 0 or df["fare_amount"][i] == False:
            deleted_row.append(i)
            
        # Trips with no distance between dropoff and pickup
        elif df["trip_distance"][i] <= 0 or df["trip_distance"][i] == False:
            deleted_row.append(i)
            
    df = df.drop(labels = deleted_row, axis=0)
    
    return df.reset_index(drop=True)

In [12]:
def get_and_clean_month_taxi_data(url):
    
    # Check if we already have the data in local, otherwise download it
    file_name = url[-31:]
    
    for i in os.listdir('./taxidata'):
        if i == file_name:
            df = pd.read_parquet(f"taxidata/{file_name}", engine='pyarrow')
            return df
    
    
    # Download and preprocessing data
    find_lat_lon = get_lat_lon_from_loc() #get the lat-lon form .shp file
    
    # Read the data from url
    response = requests.get(url, stream=True)
    with open(f"taxidata/{url[-31:]}", "wb") as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    df = pd.read_parquet(f"taxidata/{url[-31:]}", engine='pyarrow') #Reading the data
    df = df.sample(n=2564).reset_index(drop=True) #Take 2564 random sample

    try:
        #Clean the data
        df = clean_data(df)
    
        # Rename the columns
        if 'Start_Lon' in df.columns : #2009
            df = df[["Trip_Pickup_DateTime", "Start_Lon", "Start_Lat", "End_Lon", "End_Lat", "Tip_Amt"]]
            df.rename(columns={"Trip_Pickup_DateTime": 'date_time', "Start_Lon": 'pickup_longitude', "Start_Lat": 'pickup_latitude', "End_Lon": 'dropoff_longitude', "End_Lat": 'dropoff_latitude', "Tip_Amt": 'tip_amount'}, inplace=True)
        
        elif 'pickup_longitude' in df.columns : #2010
            df = df[["pickup_datetime", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "tip_amount"]]
            df.rename(columns={"pickup_datetime": 'date_time'}, inplace=True)

        else:     
            if "tpep_pickup_datetime" in df.columns : #2011-2014
                df = df[["tpep_pickup_datetime", "PULocationID", "DOLocationID", "tip_amount"]]
                df.rename(columns={"tpep_pickup_datetime": 'date_time'}, inplace=True)
        
            else: df = df[["date_time", "trip_distance", "PULocationID", "DOLocationID"]] #2015
            
            
            # Finding the lat-lon using shp file
            start_lon = []
            start_lat = []
            end_lon = []
            end_lat = []
        
        
            for i in range(len(df["PULocationID"])):
                start_point = df["PULocationID"][i]
                end_point = df["DOLocationID"][i]
            
                if df["PULocationID"][i] < 264 and df["DOLocationID"][i] < 264: #Filter for NYC Area only
                    index_location = find_lat_lon[find_lat_lon["LocationID"] == start_point].index.values[0] 
                    start_lon.append(float(find_lat_lon["lon"][index_location]))
                    start_lat.append(float(find_lat_lon["lat"][index_location]))
                
                    index_location = find_lat_lon[find_lat_lon["LocationID"] == end_point].index.values[0] 
                    end_lon.append(float(find_lat_lon["lon"][index_location]))
                    end_lat.append(float(find_lat_lon["lat"][index_location]))
                
                else: # Area outside NYC, to be deleted later
                    start_lon.append(0)
                    start_lat.append(0)
                    end_lon.append(0)
                    end_lat.append(0)
                    
        
                     
            df["pickup_longitude"] = start_lon
            df["pickup_latitude"] = start_lat
            df["dropoff_longitude"] = end_lon
            df["dropoff_latitude"] = end_lat
        
            df = df.drop(["PULocationID", "DOLocationID"], axis=1)

        
        # Filter the lat-lon between (40.560445, -74.242330) and (40.908524, -73.717047)
        df = filter_lat_lon(df)
    
        #Convert datetime str to python object
        if isinstance(df["date_time"][0], str):
            for i in df.index:
                df["date_time"][i] = datetime_str_to_obj(df["date_time"][i])
    
        # Calculate distance and add calculated_distance column
        df = add_distance_column(df)
        df= df[df['calculated_distance'] != 0]
        
        df = df[["date_time", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "tip_amount", "calculated_distance"]]
        
    except IndexError:
        os.remove(f"taxidata/{url[-31:]}")
        get_and_clean_month_taxi_data(url)
 
    
    # Re-save the file
    df.to_parquet(f"taxidata/{url[-31:]}")
    
    return df            
    

In [14]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []
    
    all_csv_urls = find_taxi_csv_urls()
    for csv_url in all_csv_urls:
        # Get and clean the data from local or url
        dataframe = get_and_clean_month_taxi_data(csv_url)
        
        # Put all the dataframe into a list
        all_taxi_dataframes.append(dataframe)
        
    # Create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes, ignore_index=True)
    
    return taxi_data

### Processing Uber Data

Functions are created to clean Uber data sets by removing trips with no passenger count and trips with no fare, as well as dropping unwanted columns and normalizing column names. This is done to facilitate more efficient and effective analysis in later steps. Details are highlighted in code comments.

In [15]:
def clean_uber_data(df):
    
    deleted_row = []
    
    for i in df.index:
        # Convert datetime str to python object
        df['pickup_datetime'][i] = df['pickup_datetime'][i].replace(' UTC', '')
        df['pickup_datetime'][i] = df['pickup_datetime'][i].replace('T', ' ')
        df['pickup_datetime'][i] = datetime_str_to_obj(df['pickup_datetime'][i])

        # Trips with zero passenger count
        if df["passenger_count"][i] < 1 or df["passenger_count"][i] == False:
            deleted_row.append(i)
            
        # Trips with no fare
        elif df["fare_amount"][i] <= 0 or df["fare_amount"][i] == False:
            deleted_row.append(i)
            
    df = df.drop(labels = deleted_row, axis=0)
    df.rename(columns={"pickup_datetime": 'date_time'}, inplace=True)

    return df.reset_index(drop=True)

In [16]:
def load_and_clean_uber_data(csv_file):
    df = pd.read_csv(csv_file)
    df = clean_uber_data(df)
    df = filter_lat_lon(df)
    # Drop unecesarry column
    df.drop(df.columns[[0, 1, 2, 8]], axis=1,inplace=True)
    return df

In [17]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_CSV)
    add_distance_column(uber_dataframe)
    uber_dataframe= uber_dataframe[uber_dataframe['calculated_distance'] != 0]
    return uber_dataframe

### Processing Weather Data

Functions are created to clean weather data by dropping unwanted columns and normalizing column names. Note that we have split the whole weather data into two separate sets for hourly and daily data respectively. In addition, we have specially curated the precipitation column to account for missing daily precipitation data by using the sum of hourly precipitation to estimate daily precipitation This is done to facilitate more efficient and effective analysis in later steps. Details are highlighted in code comments.

In [18]:
def clean_month_weather_data_hourly(csv_file):
    df2 = pd.read_csv('weatherdata/' + csv_file)
    df2 = df2[['DATE', 'HourlyPrecipitation', 'HourlyWindSpeed']]
    
    for i in df2.index:
        # Convert datetime str to python object
        df2['DATE'][i] = df2['DATE'][i].replace('T', ' ')
        df2['DATE'][i] = datetime_str_to_obj(df2['DATE'][i])
    
        try:
            df2["HourlyPrecipitation"][i] = float(df2["HourlyPrecipitation"][i])       
            if(df2["HourlyPrecipitation"][i] <= 0 or df2["HourlyPrecipitation"][i] == False):
                df2["HourlyPrecipitation"][i] = None

        except ValueError:
            df2["HourlyPrecipitation"][i] = None
            
            
        try:
            df2["HourlyWindSpeed"][i] = int(df2["HourlyWindSpeed"][i])
            if (df2["HourlyWindSpeed"][i] <= 0 or df2["HourlyWindSpeed"][i] == False):
                df2["HourlyWindSpeed"][i] = None
            
        except ValueError:
            df2["HourlyWindSpeed"][i] = None
       
    
    return df2.reset_index(drop=True)

In [ ]:
def clean_month_weather_data_daily(csv_file):
    df3 = pd.read_csv('weatherdata/' + csv_file)

    df3 = df3[['DATE', 'DailyAverageWindSpeed', 'DailyPrecipitation', 'Sunrise', 'Sunset']]
    df3 = df3.dropna(thresh=3, how='all')
    
    
    for i in df3.index:
        # Convert datetime str to python object
        df3['DATE'][i] = df3['DATE'][i].replace('T', ' ')
        df3['DATE'][i] = datetime_str_to_obj(df3['DATE'][i])
        
        # Convert the sunrise sunset data to int
        df3["Sunrise"][i] = int(df3["Sunrise"][i])
        df3["Sunset"][i] = int(df3["Sunset"][i])
        
        try:
            df3["DailyPrecipitation"][i] = float(df3["DailyPrecipitation"][i])
            if(df3["DailyPrecipitation"][i] <= 0 or df3["DailyPrecipitation"][i] == False):
                df3["DailyPrecipitation"][i] = None
        except ValueError:
            df3["DailyPrecipitation"][i] = None
            
        try:
            df3["DailyAverageWindSpeed"][i] = float(df3["DailyAverageWindSpeed"][i])
            if(df3["DailyAverageWindSpeed"][i] <= 0 or df3["DailyAverageWindSpeed"][i] == False):
                df3["DailyAverageWindSpeed"][i] = None
        except ValueError:
            df3["DailyAverageWindSpeed"][i] = None
    
    
    return df3.reset_index(drop=True)

In [19]:
def curated_daily_data(hourly_weather_data, daily_weather_data):
    # Get the date from the hourly data
    hourly_weather_data['h_temp'] = hourly_weather_data.apply(lambda row: row["DATE"].strftime("%Y-%m-%d") , axis=1)
    
    # Estimate the daily precipitation and windspeed from hourly data
    hourly_weather_data['prep_temp'] = hourly_weather_data.apply(lambda row: hourly_weather_data.loc[hourly_weather_data['h_temp'] == row['h_temp'], 'HourlyPrecipitation'].sum() , axis=1)
    hourly_weather_data['wind_temp'] = hourly_weather_data.apply(lambda row: hourly_weather_data.loc[hourly_weather_data['h_temp'] == row['h_temp'], 'HourlyWindSpeed'].mean() , axis=1)

    # Drop the duplicates date and put it into temporary dataframe
    df_temp = hourly_weather_data[["h_temp", "prep_temp", "wind_temp"]].drop_duplicates(subset = ["h_temp"]).reset_index(drop=True)

    # Get only the date from the daily data
    daily_weather_data['d_temp'] = daily_weather_data.apply(lambda row: row["DATE"].strftime("%Y-%m-%d") , axis=1)
    
    # Check if the precipitation and windspeed from daily data is zero or null, it will use the estimated value from hourly data 
    for i in daily_weather_data.index:
        if daily_weather_data["DailyAverageWindSpeed"][i] and daily_weather_data["DailyAverageWindSpeed"][i] > 0:
            index = df_temp.index[df_temp['h_temp'] == daily_weather_data["d_temp"][i]]
        
            df_temp["wind_temp"][i] = daily_weather_data["DailyAverageWindSpeed"][i]
    
        if daily_weather_data["DailyPrecipitation"][i] and daily_weather_data["DailyPrecipitation"][i] > 0:
            index = df_temp.index[df_temp['h_temp'] == daily_weather_data["d_temp"][i]]
        
            df_temp["prep_temp"][i] = daily_weather_data["DailyPrecipitation"][i]
                
    
    return df_temp
    

In [20]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files or just add the name/paths manually
    weather_csv_files = ["2009_weather.csv", "2010_weather.csv", "2011_weather.csv", "2012_weather.csv", "2013_weather.csv", "2014_weather.csv", "2015_weather.csv"]


    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes, ignore_index=True)
    daily_data = pd.concat(daily_dataframes, ignore_index=True)
    
    
    daily_sunrisesunset_data = daily_data[["DATE", "Sunrise", "Sunset"]]
    
    daily_sunrisesunset_data['DATE'] = daily_sunrisesunset_data.apply(lambda row: row["DATE"].strftime("%Y-%m-%d") , axis=1)
    
    for i in daily_sunrisesunset_data.index:
        x = str(daily_sunrisesunset_data["Sunrise"][i])
        daily_sunrisesunset_data["Sunrise"][i] = f"{x[:-4]}:{x[-4:-2]}"
        daily_sunrisesunset_data["Sunrise"][i] = datetime.datetime.strptime(daily_sunrisesunset_data["Sunrise"][i], '%H:%M').time()
        y = str(daily_sunrisesunset_data["Sunset"][i])
        daily_sunrisesunset_data["Sunset"][i] = f"{y[:-4]}:{y[-4:-2]}"
        daily_sunrisesunset_data["Sunset"][i] = datetime.datetime.strptime(daily_sunrisesunset_data["Sunset"][i], '%H:%M').time()
    
    # Curated the daily data vs hourly data
    new_daily_data = curated_daily_data(hourly_data, daily_data)  
    
    # Normalize the column name
    hourly_data = hourly_data[["DATE", "HourlyPrecipitation", "HourlyWindSpeed"]]
    new_daily_data.rename(columns={"h_temp": 'DATE', "prep_temp": 'DailyPrecipitation', "wind_temp": 'DailyAverageWindSpeed' }, inplace=True)
    
    
    return hourly_data, new_daily_data, daily_sunrisesunset_data
    

### Process All Data

All the required functions written above are executed to obtain all the cleaned data sets used for analysis.

In [ ]:
taxi_data = get_and_clean_taxi_data()

str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee
str nee


In [ ]:
# Filter taxi data
taxi_data = taxi_data[["date_time", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "calculated_distance", "tip_amount"]]


In [ ]:
uber_data = get_uber_data()

In [ ]:
hourly_weather_data, daily_weather_data, daily_sunrisesunset_data = load_and_clean_weather_data()

In [ ]:
# TO BE DELETED, READ DATA FROM EXCEL

#a = pd.read_excel('taxi_data.xlsx')
#b = pd.read_excel('uber_data.xlsx')
#c = pd.read_excel('hourly_weather_data.xlsx')
#d = pd.read_excel('daily_weather_data.xlsx')
#e = pd.read_excel('daily_sunrisesunset_data.xlsx')

## Part 2: Storing Cleaned Data

Using Python's `sqlite3` module, we create a database simply by creating a connection to it and then add information to our database. This is done by creating tables and specifying the schema for the table. 

In [ ]:
import sqlite3
connection = sqlite3.connect("final_project.db")
connection

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATETIME,
    LATITUDE FLOAT,
    LONGITUDE FLOAT,
    HourlyPrecipitation FLOAT,
    HourlyWindSpeed FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    LATITUDE FLOAT,
    LONGITUDE FLOAT,
    DailyPrecipitation FLOAT,
    DailyAverageWindSpeed FLOAT,
    Sunrise FLOAT,
    Sunset FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips
(
    id INTEGER PRIMARY KEY,
    date_time DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    tip_amount FLOAT,
    calculated_distance FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips
(
    id INTEGER PRIMARY KEY,
    date_time DATETIME,
    pickup_longitude FLOAT,
    pickup_latitude FLOAT,
    dropoff_longitude FLOAT,
    dropoff_latitude FLOAT,
    calculated_distance FLOAT
);
"""

DAILY_SUNRISE_SUNSET_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_sunrise_sunset
(
    id INTEGER PRIMARY KEY,
    DATE DATE,
    Sunrise TIMESTAMP,
    Sunset TIMESTAMP,
    FOREIGN KEY(DATE) REFERENCES daily_weather(DATE)
);
"""

In [ ]:
# create that required schema.sql file
with open('sql_files/schema.sql', "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)
    f.write(DAILY_SUNRISE_SUNSET_SCHEMA)

In [ ]:
# create the tables with the schema files
with connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
with connection:
    connection.execute(DAILY_WEATHER_SCHEMA)
with connection:
    connection.execute(TAXI_TRIPS_SCHEMA)
with connection:
    connection.execute(UBER_TRIPS_SCHEMA)
with connection:
    connection.execute(DAILY_SUNRISE_SUNSET_SCHEMA)

### Add Data to Database

Data is added to a SQLite Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    for key, value in table_to_df_dict.items():
        print(key)
        value.to_sql(name=key, con=connection, if_exists='append', index=False)
    
    return 'Success adding data'

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
    "daily_sunrise_sunset": daily_sunrisesunset_data
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as f:
        f.write(query)
    
    return f'Succes generate {outfile}'

### Query 1

Constructs and executes a query to categorize the number of yellow taxi trips for each hour of the day, in descending order. This is to determine which hour of the day was the most popular to take a yellow taxi, from 01-2009 through 06-2015. 

In [ ]:
QUERY_1 = """
SELECT 
    DISTINCT strftime('%H', date_time) AS time,
    COUNT (*) as trip
FROM taxi_trips
WHERE date_time between '2009-01-01' AND '2015-06-30'
GROUP BY time
ORDER BY trip DESC
"""

In [ ]:
with connection:
    result = connection.execute(QUERY_1)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_1, "sql_files/taxi_trips_numbers_by_hour_in_descending_order.sql")

### Query 2

Constructs and executes a query to categorize the number of uber trips for each day of the week, in descending order. This is to determine what day of the week was the most popular to take an uber.  

In [ ]:
QUERY_2 = """
SELECT  case cast (strftime('%w', date_time) as integer)
  when 0 then 'Sunday'
  when 1 then 'Monday'
  when 2 then 'Tuesday'
  when 3 then 'Wednesday'
  when 4 then 'Thursday'
  when 5 then 'Friday'
  else 'Saturday' end as day,
  COUNT(*) as no_of_trip
FROM uber_trips
WHERE date_time between '2009-01-01' AND '2015-06-30'
GROUP BY day
ORDER BY no_of_trip DESC
"""

# WHERE tpep_pickup_datetime between '2009-01-01' AND '2015-06-30'

In [ ]:
with connection:
    result = connection.execute(QUERY_2)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_2, "sql_files/uber_trips_numbers_for_each_day_in_descending_order.sql")

### Query 3

Constructs and executes a query to find the 95% percentile of distance travelled for all hired trips during July 2013.

In [ ]:
QUERY_3 = """
WITH 
base 
AS (
    SELECT
        date,
        calculated_distance,
        ROW_NUMBER() OVER(ORDER BY calculated_distance ASC) AS row_num
    FROM (
        SELECT date(date_time) as date, calculated_distance
        FROM taxi_trips
        WHERE date between '2013-07-01' AND '2013-07-31'
        UNION ALL
        SELECT date(date_time) as date, calculated_distance
        FROM uber_trips
        WHERE date between '2013-07-01' AND '2013-07-31'
    )
    WHERE date between '2013-07-01' AND '2013-07-31'
    ),
    
quantile
AS (
    SELECT
        round(0.95 * COUNT(calculated_distance)) AS n_quantile
    FROM
        base
    )
    
select 
base.calculated_distance 
from base
join quantile
on base.row_num = quantile.n_quantile
"""

In [ ]:
with connection:
    result = connection.execute(QUERY_3)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_3, "sql_files/95%_percentile_of_distance_travelled_for_all_hired_trips_July_2013.sql")

### Query 4

Constructs and executes a query to categorize the top 10 days with the greatest number of hired rides, and the corresponding average distance for each day. 

In [ ]:
QUERY_4 = """
SELECT
    date,
    COUNT (*) as no_of_trip,
    AVG (calculated_distance)
FROM (
    SELECT date(date_time) AS date, calculated_distance
    FROM taxi_trips
    UNION ALL
    SELECT date(date_time) AS date, calculated_distance
    FROM uber_trips
)
WHERE date between '2009-01-01' AND '2009-12-31'
GROUP BY date
ORDER BY no_of_trip DESC
LIMIT 10
"""

In [ ]:
# TOBEDELETED Read Data
with connection:
    result = connection.execute(QUERY_4)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_4, "sql_files/top_10_days_with_greatest_number_hired_rides_and_corresponding_average_distance.sql")

### Query 5

Constructs and executes a query to determine the top 10 windiest days in 2014 on average, and the number of hired trips made on those days. 

In [ ]:
QUERY_5 = """
WITH
weather AS (
SELECT
    date(DATE) as date,
    DailyAverageWindSpeed
FROM
    daily_weather
WHERE date between '2014-01-01' AND '2014-12-31'
ORDER BY DailyAverageWindSpeed DESC
LIMIT 10),

trip AS (
SELECT
    date,
    COUNT (*) as no_of_trip
FROM (
    SELECT date(date_time) AS date
    FROM taxi_trips
    WHERE date between '2014-01-01' AND '2014-12-31'
    UNION ALL
    SELECT date(date_time) AS date
    FROM uber_trips
    WHERE date between '2014-01-01' AND '2014-12-31'
)
GROUP BY date
)


SELECT
    weather.*,
    trip.no_of_trip
FROM
    weather
LEFT JOIN trip
ON weather.date = trip.date
"""

In [ ]:
with connection:
    result = connection.execute(QUERY_5)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_5, "sql_files/top_10_windiest_days_2014_and_corresponding_number_of_hired_trips.sql")

### Query 6

Constructs and executes a query to determine during hurricane Sandy in NYC (Oct 29-30, 2012), plus the week leading up and the week after, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive, and what was the sustained wind speed. 

In [ ]:
QUERY_6 = """
WITH
weather AS (
SELECT
    strftime('%Y-%m-%d', DATE) AS date,
    strftime('%H', DATE) AS time,
    sum(HourlyPrecipitation) as Precipitation,
    avg(HourlyWindSpeed) as Wind_Speed
FROM
    hourly_weather
WHERE date between '2012-10-22' AND '2012-11-5'
GROUP BY time
),

trip AS (
SELECT
    date,
    time,
    COUNT (*) as no_of_trip
FROM(
    SELECT 
        date(date_time) AS date, 
        strftime('%H', date_time) AS time
    FROM taxi_trips
    WHERE date between '2012-10-22' AND '2012-11-5'
    UNION ALL
    SELECT 
        date(date_time) AS date, 
        strftime('%H', date_time) AS time
    FROM uber_trips
    WHERE date between '2012-10-22' AND '2012-11-5'
)
GROUP BY date, time
)


SELECT
    weather.date,
    weather.time,
    trip.no_of_trip,
    weather.Precipitation,
    weather.Wind_Speed
FROM
    weather
LEFT JOIN trip
ON weather.time = trip.time AND weather.date = trip.date
ORDER BY weather.date
"""

In [ ]:
with connection:
    result = connection.execute(QUERY_6)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_6, "sql_files/Hurricane_Sandy_tripsNumber_precipitation_windspeed_byHour.sql")

### Query 7 (Extra Credit)

Constructs and executes a query to determine what are the top 10 sunrise times with highest average wind speed?

In [1]:
QUERY_EXTRA = """
SELECT
    strftime('%H:%M', x.Sunrise) AS time,
    avg(y.DailyAverageWindSpeed) AS average_windspeed
FROM
    daily_sunrise_sunset AS x
JOIN daily_weather AS Y
ON x.id = Y.id
GROUP BY time
ORDER BY average_windspeed DESC
LIMIT 10
"""

In [ ]:
with connection:
    result = connection.execute(QUERY_EXTRA)

for row in result:
    print(row)

In [ ]:
write_query_to_file(QUERY_EXTRA, "sql_files/top_10_sunrise_times_with_highest_average_windspeed.sql")

## Part 4: Visualizing the Data

### Visualization 1

A bar chart is created to depict the total number of yellow taxi trips for each hour, from for 01-2009 through 06-2015. Data value labels are added for greater visualization. We chose to use bar charts for visualization to show segments of information and make comparisons most effectively. From the bar chart, 1900 hours was the most popular to take a yellow taxi. 

In [ ]:
def addlabels(x,y):
        for i in range(len(x)):
            plt.text(i,y[i],y[i])

In [ ]:
def plot_visual_1(data):
    # preparing the dataset
    hours = list(data.keys())
    trips = list(data.values())

    fig = plt.figure(figsize = (10, 5))
 
    # creating the bar plot
    bars = plt.bar(hours, trips, color ='lightblue', width = 0.4)
 
    plt.xlabel("Hour", fontsize = 13)
    plt.ylabel("No. of trip", fontsize = 13)
    plt.title("No of yellow taxi trip in Hour basis", fontsize = 13)
    
    plt.yticks(fontsize=10)
    plt.xticks(fontsize=10)

 
    
            
    addlabels(hours, trips)

    
    plt.show()
    
    highest = max(trips)
    index = trips.index(highest)
    
    print(f"the hour that was the most popular to take yellow taxi is {hours[index]}")

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    QUERY = """
    SELECT 
        DISTINCT strftime('%H', date_time) AS time,
        COUNT (*) as trip
    FROM taxi_trips
    WHERE date_time between '2009-01-01' AND '2015-06-30'
    GROUP BY time
    ORDER BY time ASC
    """
    
    with connection:
        output = connection.execute(QUERY)
    
    data = {}
    
    for row in output:
        data[str(row[0])] = row[1]
        
     
    return data

In [ ]:
qurated_taxidata = get_data_for_visual_1()
plot_visual_1(qurated_taxidata)

## Visualization 2

A scatter plot is created to shows the average distance travelled per month for both taxis and Ubers combined. In addition, a 90% confidence interval, highlighted in blue, is created around the mean.

In [ ]:
import statistics
from math import sqrt

def plot_visual_2(data):
    
    month = list(data.keys())
    avg_distance = list(data.values())
    z = 1.645
    
    stdev = statistics.stdev(avg_distance)
    confidence_interval = z * stdev / sqrt(len(avg_distance))
    
    figure = plt.figure(figsize = (10, 5))
    
    top = [i + confidence_interval for i in avg_distance]
    bottom = [i - confidence_interval for i in avg_distance]

    
 
    plt.plot(month, avg_distance,'o', color='red')
    plt.fill_between(month, bottom, top, color = 'lightblue', alpha = 0.5)
    
    
    addlabels(month, avg_distance)
    
    plt.xlabel("Month")
    plt.ylabel("Average Distance")
    plt.title("")
    
    plt.show()

In [ ]:
def get_data_for_visual_2():
    
    QUERY = """   
    SELECT
        month,
        round(AVG (calculated_distance), 2)
    FROM (
        SELECT strftime('%m', date_time) AS month, calculated_distance
        FROM taxi_trips
        UNION ALL
        SELECT strftime('%m', date_time) AS month, calculated_distance
        FROM uber_trips
    )   
    GROUP BY month
    """
    
    with connection:
        output = connection.execute(QUERY)
    
    data = {} 
    
    for row in output:
        data[str(row[0])] = row[1]
        
    return data

In [ ]:
visualization2_dataframe = get_data_for_visual_2()
plot_visual_2(visualization2_dataframe)

## Visualization 3

A bar chart is created to compare which day of the week was most popular for drop offs for the three major New York airports: LGA, JFK, and EWR. The website bboxfinder was used to obtain the coordinate boxes for the three airports. A Bar chart was chosen to show segments of information and make comparisons most effectively.

The boxes for all airports was obtained from bboxfinder are:

LGA = ((40.785860, -73.902975), (40.759019, -73.856576))
JFK = ((40.676904, -73.824550), (40.623357, -73.741224))
EWR = ((40.712517, -74.192865), (40.660366, -74.149802))

The above coordinate will bse used in the SQL Query for this question

In [ ]:
# use a more descriptive name for your function
def plot_visual_3(day, airport, trip):
    
    labels = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
    LGA = []
    JFK = []
    EWR = []
    
    for i in range(len(day)):
        for j in labels:
            if day[i] == j:
                if airport[i] == 'LGA':
                    LGA.append(trip[i])
                elif airport[i] == 'JFK':
                    JFK.append(trip[i])
                elif airport[i] == 'EWR':
                    EWR.append(trip[i]) 
    
    x = np.arange(len(labels))  # the label locations
    width = 0.15  # the width of the bars

    fig, ax = plt.subplots(figsize=(10,6))
    rects1 = ax.bar(x - width, LGA, width, label='LGA')
    rects2 = ax.bar(x + width, JFK, width, label='JFK')
    rects3 = ax.bar(x, EWR, width, label='EWR')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('No of Trip', fontsize = 12)
    ax.set_title('No of Trip based on Day')
    ax.set_xticks(x, labels, fontsize = 12)
    ax.legend(loc='upper right', fontsize = 12)
    plt.ylim(0, 2500)
    plt.rc('ytick', labelsize=10) 

    ax.bar_label(rects1, padding=3, fontsize = 12)
    ax.bar_label(rects2, padding=3, fontsize = 12)
    ax.bar_label(rects3, padding=3, fontsize = 12)

    fig.tight_layout()

    plt.show()

In [ ]:
def get_data_for_visual_3():
    # Query SQL database for the data needed.
    QUERY = """
    SELECT
    day_of_week,
    Airport,
    SUM(number_of_trip) AS total_trip
    FROM (
    SELECT
        CASE CAST(STRFTIME('%w', date_time) AS integer)
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        WHEN 6 THEN 'Saturday'
        END AS day_of_week,
        CASE 
            WHEN (dropoff_longitude BETWEEN -73.902975 AND -73.856576) 
            AND (dropoff_latitude BETWEEN 40.759019 AND 40.785860)
            THEN 'LGA'
            WHEN (dropoff_longitude BETWEEN -73.824550 AND -73.741224)
            AND (dropoff_latitude BETWEEN 40.623357 AND 40.676904)
            THEN 'JFK'
            WHEN (dropoff_longitude BETWEEN -74.192865 AND -74.149802)
            AND (dropoff_latitude BETWEEN 40.660366 AND 40.712517)
            THEN 'EWR'
        END AS 'Airport',
        COUNT(*) AS number_of_trip        
        
    FROM taxi_trips
    GROUP BY 1,2
    
    UNION ALL
    
    SELECT
        CASE CAST(STRFTIME('%w', date_time) AS integer)
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        WHEN 6 THEN 'Saturday'
        END AS day_of_week,
        CASE 
            WHEN (dropoff_longitude BETWEEN -73.902975 AND -73.856576) 
            AND (dropoff_latitude BETWEEN 40.759019 AND 40.785860)
            THEN 'LGA'
            WHEN (dropoff_longitude BETWEEN -73.824550 AND -73.741224)
            AND (dropoff_latitude BETWEEN 40.623357 AND 40.676904)
            THEN 'JFK'
            WHEN (dropoff_longitude BETWEEN -74.192865 AND -74.149802)
            AND (dropoff_latitude BETWEEN 40.660366 AND 40.712517)
            THEN 'EWR'
        END AS 'Airport',
        COUNT(*) AS number_of_trip        
        
    FROM uber_trips
    GROUP BY 1,2
    )
    WHERE Airport IN ('JFK', 'LGA', 'EWR')
    GROUP BY 1,2
    """
    
    # You can put the data queried into a pandas dataframe, if you wish
    with connection:
        output = connection.execute(QUERY)
    
    day = []
    airport = []
    trip = []
    
    for row in output:
        day.append(row[0])
        airport.append(row[1])
        trip.append(row[2])
        
    return day, airport, trip

In [ ]:
day, airport, trip = get_data_for_visual_3()
plot_visual_3(day, airport, trip)

## Visualization 4

KeplerGL is used to create a geospatial visualization of the heatmap of all hired trips over a map of the bounded New York area.

In [ ]:
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd



In [ ]:
map = KeplerGl(height=600, width=800)
#show the map
map.add_data(data=df4,name='New York City Taxi Trips')
map.add_data(data=df5,name='New York City Uber Trips')
map

## Visualization 5
A scatter plot is created to compare tip amount vs distance for Yellow Taxi rides. We chose to use a scatter plot in order to best depict the relationship between the two variables. A line of best fit is also added. The general trend is that the is a positive relationship between tip amount and distance. 

In [ ]:
import numpy as np

def plot_visual_5(data):
    # preparing the dataset
    tips = list(data.keys())
    
    distance = list(data.values())

    fig = plt.figure(figsize = (10, 5))
 
    #Creating the bar plot
    plt.plot(tips, distance,'o', color='#f44336')
 
    plt.xlabel("Tip Amount", fontsize = 13)
    plt.ylabel("Distance", fontsize = 13)
    plt.title("Graph of Tip Amount versus Distance for Yellow Taxi rides", fontsize = 13)
    
    plt.yticks(fontsize=10)
    plt.xticks(fontsize=10)
    
    x = np.array(tips)
    y = np.array(distance)
    a, b = np.polyfit(x, y, 1)

    #Creating the line of best fit 
    plt.plot(x, a*x+b)

    plt.show()

In [ ]:
def get_data_for_visual_5():
    # Query SQL database for the data needed.
    QUERY_V5 = """
    SELECT 
        tip_amount,
        calculated_distance
    FROM (taxi_trips
    )
    WHERE tip_amount IS NOT NULL AND calculated_distance IS NOT NULL AND tip_amount BETWEEN 0 and 40
    """
    
    with connection:
        output = connection.execute(QUERY_V5)
    
    data = {}
    
    for row in output:
        data[row[0]] = row[1]
 
    return data

In [ ]:
visualization5_dataframe = get_data_for_visual_5()
plot_visual_5(visualization5_dataframe)